# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv


In [2]:
import os
import pandas as pd

PRICE_DATA = os.getenv('PRICE_DATA')

PRICE_DATA

'../../05_src/data/prices/'

In [3]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.
parquet_files = glob(os.path.join(PRICE_DATA, '*/*.parquet'))
dd_px = dd.read_parquet(PRICE_DATA).set_index("ticker")



In [5]:
type(parquet_files)

list

In [6]:
dd_px.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.613003,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.281467,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.782791,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.344173,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.372852,2819626,Health Care,Life Sciences Tools & Services,2000


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
# Write your code below.
import numpy as np

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0)*1
).assign(
    hi_lo_range = lambda x: (x['High']-x['Low'])
))


/var/folders/qj/5jhw2kb54ybghqyjtqgks9yr0000gn/T/ipykernel_59249/1757173458.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [8]:
type(dd_feat)

dask_expr._collection.DataFrame

In [9]:
x_dt = dd_feat.compute()
x_dt

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,positive_return,hi_lo_range
ticker,,,,,,,,,,,,,,
OXY,2002-01-02,12.670362,12.742217,12.440428,12.699104,6.993296,2083374,Energy,Oil & Gas Exploration & Production,2002,NaN,NaN,0,0.301789
OXY,2002-01-03,12.794910,12.794910,12.550605,12.641620,6.961638,2154768,Energy,Oil & Gas Exploration & Production,2002,12.699104,-0.004527,0,0.244306
OXY,2002-01-04,12.584137,12.770959,12.550605,12.747007,7.019673,1297412,Energy,Oil & Gas Exploration & Production,2002,12.641620,0.008337,1,0.220354
OXY,2002-01-07,12.670362,12.842813,12.560185,12.703894,6.995931,1525581,Energy,Oil & Gas Exploration & Production,2002,12.747007,-0.003382,0,0.282627
OXY,2002-01-08,12.713475,12.713475,12.478750,12.512282,6.890413,1186354,Energy,Oil & Gas Exploration & Production,2002,12.703894,-0.015083,0,0.234725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INTU,2023-12-22,622.830017,625.150024,617.679993,624.070007,622.262207,820800,Information Technology,Application Software,2023,620.989990,0.004960,1,7.470032
INTU,2023-12-26,625.169983,628.330017,622.729980,624.849976,623.039978,638300,Information Technology,Application Software,2023,624.070007,0.001250,1,5.600037
INTU,2023-12-27,623.989990,629.799988,622.260010,629.119995,627.297607,734400,Information Technology,Application Software,2023,624.849976,0.006834,1,7.539978


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [10]:
# Write your code below.
type(x_dt)


pandas.core.frame.DataFrame

In [12]:
x_dt['10_avg_return'] = x_dt['returns'].rolling(window=10).mean()

print(x_dt.head(20))

             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
OXY    2002-01-02  12.670362  12.742217  12.440428  12.699104   6.993296   
OXY    2002-01-03  12.794910  12.794910  12.550605  12.641620   6.961638   
OXY    2002-01-04  12.584137  12.770959  12.550605  12.747007   7.019673   
OXY    2002-01-07  12.670362  12.842813  12.560185  12.703894   6.995931   
OXY    2002-01-08  12.713475  12.713475  12.478750  12.512282   6.890413   
OXY    2002-01-09  12.454798  12.588927  12.363783  12.478750   6.871951   
OXY    2002-01-10  12.454798  12.560185  12.392524  12.473960   6.869311   
OXY    2002-01-11  12.502702  12.502702  12.119477  12.167380   6.700480   
OXY    2002-01-14  12.167380  12.263186  12.061993  12.162590   6.697839   
OXY    2002-01-15  12.282347  12.368573  12.248815  12.335041   6.792805   
OXY    2002-01-16  12.335041  12.335041  11.975768  12.004510   6.610786   
OXY    2002-

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?\
No, it is not necessary to convert the Dask DataFrame to a Pandas DataFrame, we can calculate the rolling average directly on Dask DataFrames.


+ Would it have been better to do it in Dask? Why?\
Yes, it would have been better to perform the calculation in Dask.\
As we discuss in class, Dask is designed for large datasets with parallelizing operations and distributed conputations, so that we could save memory and it's mor efficient and faster.\
Also, Dask is a lazy execution framework: commands will not execute until they are required. 

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.